# Ayudantía 6: Interfaces Gráficas 2
### Autores:
 - Sofía Arratia Arriaza (@Sofia-Arratia)
 - Francisca Cares (@franciscares)
 - Pablo Kipreos Palau (@Pablok98)
 - Cristobal Pérez-Cotapos (@CristobalPerez-Cotapos)


## Ventana personalizada
Ustedes podrán construir sus propias ventanas personalizadas. Para esto, simplemente tienen que heredar QWidget (o cualquier otra clase que quieran personalizar, como botones). El ejemplo anterior puede ser programado con un QWidget personalizado:

In [ ]:
import sys
from PyQt5.QtWidgets import QWidget, QApplication

class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.setGeometry(200, 100, 300, 300)
        self.setWindowTitle('Ventana personalizada bien bacan')


if __name__ == '__main__':
    app = QApplication([])
    ventana = MiVentana()
    ventana.show()
    sys.exit(app.exec_())

### ¿Como interactúa nuestra ventana con el programa?
## Eventos y señales 🙌

In [ ]:
from PyQt5.QtCore import (pyqtSignal)

class Clase(QObject): # Recuerda heredar QObject!
    signal = pyqtSignal(alguna_clase)

- Ventana $\Large\rightarrow$ Programa

- Ventana $\Large\leftarrow$ Programa

### Ejemplo conexión entre ventanas

In [ ]:
import sys
from PyQt5.QtWidgets import QWidget, QApplication, QPushButton, QLineEdit, QLabel
from PyQt5.QtCore import pyqtSignal


class MiVentanaBotones(QWidget):
    senal_texto = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.setGeometry(200, 100, 300, 300)
        self.setWindowTitle('Ventana personalizada con botones bien bacanes')

        self.boton_print_1 = QPushButton('Mensaje bonito', self)
        self.boton_print_2 = QPushButton('Mensaje no tan bonito', self)

        self.ini_gui()

In [ ]:
    def ini_gui(self):
        self.boton_print_1.setGeometry(50, 0, 150, 50)
        self.boton_print_2.setGeometry(50, 100, 150, 50)

        self.boton_print_1.clicked.connect(self.enviar_texto)
        self.boton_print_2.clicked.connect(self.enviar_texto)

        self.show()

In [ ]:
    def enviar_texto(self):
        boton_clickeado = self.sender()
        texto_boton = boton_clickeado.text()
        self.senal_texto.emit(texto_boton)

In [ ]:
class MiVentanaTexto(QWidget):
    def __init__(self):
        super().__init__()
        self.setGeometry(600, 100, 300, 300)
        self.setWindowTitle('Ventana personalizada con texto bien bacan')
        self.label_mensaje = QLabel('La bola de cristal dice: ', self)

        self.ini_gui()

In [ ]:
    def ini_gui(self):
        self.label_mensaje.move(25, 100)
        self.show()

    def imprimir_texto(self, texto):
        self.label_mensaje.setText(texto)

In [ ]:
if __name__ == '__main__':
    app = QApplication([])
    ventana_botones = MiVentanaBotones()
    ventana_texto = MiVentanaTexto()

    ventana_botones.senal_texto.connect(ventana_texto.imprimir_texto)
    sys.exit(app.exec_())

## Threading y PyQt 😱

### Espera... **¿Otra vez threading?** Esto tiene que ser una broma.
El uso de *threading* al programar interfaces gráficas es muy importante. Solo imagina un mundo donde tu navegador de internet pudiera solo manejar
una ventana o proceso a la vez: abres una ventana nueva y las otras se congelan. ¿A nadie le gustaría eso, o sí?

### **QThread** y **QTimer** al rescate
 - PyQt trae su propia implementación de *threads*, por medio de la clase llamada ``QThread``. Te recomendamos fuertemente utilizarla siempre que necesites threading en PyQt, pues te ahorrarás muchos dolores de cabeza.
Se usan de manera muy parecida a los *threads* que ya conoces y **amas**, además de tener nuevas funcionalidades :D

 - Por otro lado, una herramienta **muy útil** de PyQt para simular concurrencia son los ``QTimer`` (en este caso, *no* es lo mismo que un *timer* normal de Python).
 
 - Un ``Qtimer`` se ejecuta periódicamente, esperando un intervalo de tiempo definido entre ciclos. La forma en que se comportan los ``QTimer`` es ideal para cualquier funcionalidad que quieras que ocurra cada cierto tiempo, como veremos en el ejemplo de esta sección.

#### Métodos notables de QThread
 - ``isRunning``: reemplaza el método ``is_alive`` de los *threads* de Python. Permite saber si un ``QThread`` está actualmente corriendo o no.

#### Métodos notables de QTimer
 - ``start`` y ``stop``: permite iniciar y parar el *timer*, respectivamente.

 - ``setInterval(ms: int)``: define que el *timer* debe emitir la señal *timeout* cada ``ms`` milisegundos.
 - ``timeout``: es la señal que llama el *timer* cuando termina el intervalo de tiempo. Puedes utilizar el método ``connect`` para conectarlo a alguna función.

 - ``isActive``: permite saber si el *timer* está actualmente corriendo (análogo a ``isRunning`` e ``is_alive``).
 - ``setSingleShot(singleShoot: bool)``: permite definir si el *timer* es de tipo ``singleShoot`` (entregando como parámetro ``True``). Que un *timer* sea ``singleShoot`` significa que, al pasar el intervalo de tiempo,
 el timer se detendrá (es decir, no cicla indefinidamente).

### La clave del éxito: ¡Señales!
Hasta ahora, los ``QThreads`` (o *threads* en general) parecen algo que solo utilizarías si te lo piden explícitamente en la tarea... pero, en la práctica, es casi imposible implementar interfaces gráficas sin *threading*.

Una de las cosas más útiles que podemos hacer con ``QThreads`` es enviar señales entre ventanas u objetos, sin que se congelen o dejen de hacer sus respectivas funcionalidaes. ¡Veamos un ejemplo!

#### Primero, intentemos hacer un *loop* dentro de una ventana

In [1]:
# Importacion de librerias para todas las celdas del ejemplo
import sys
from time import sleep
from PyQt5.QtCore import pyqtSignal, QThread, QTimer
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton

In [ ]:
class VentanaSinThread(QWidget):
    actualizar_label_signal = pyqtSignal()

    def __init__(self):
        super().__init__()
        # Creamos los botones y labels necesarios para el ejemplo.
        self.label_numero = QLabel("0", self)  # Muestra el numero que ira en aumento
        self.boton_numero = QPushButton("0", self)  # Muestra el numero que sube si lo apretamos
        self.boton_loop = QPushButton("Iniciar Loop", self)  # Inicia el loop
        self.layout_principal = QVBoxLayout(self)  # Layout de la ventana principal

        self.init_gui()

In [ ]:
    def init_gui(self):
        # Ordenamos las Widgets
        self.layout_principal.addWidget(self.label_numero)
        self.layout_principal.addStretch()
        self.layout_principal.addWidget(self.boton_numero)
        self.layout_principal.addWidget(self.boton_loop)
        # Conectamos las senales
        self.boton_numero.clicked.connect(self.actualizar_boton)
        self.boton_loop.clicked.connect(self.iniciar_loop)
        self.actualizar_label_signal.connect(self.actualizar_label)

        self.show()

In [ ]:
    def actualizar_label(self):
        # Obtenemos el numero actual del label y lo aumentamos en 1
        numero_actual = int(self.label_numero.text())
        self.label_numero.setText(str(numero_actual + 1))

    def actualizar_boton(self):
        # Obtenemos el numero actual del boton y lo aumentamos en 1
        numero_actual = int(self.boton_numero.text())
        self.boton_numero.setText(str(numero_actual + 1))

    def iniciar_loop(self):
        # Emitimos la senal 10 veces, con 0.5 segundos de espera entre emisiones.
        for _ in range(10):
            self.actualizar_label_signal.emit()
            sleep(0.5)

In [2]:
if __name__ == '__main__':
    app = QApplication([])
    ventana = VentanaSinThread()
    #ventana = VentanaConThread()
    #ventana = VentanaConTimer()
    sys.exit(app.exec_())

SystemExit: 0

c:\users\pjkip\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### ¿Qué pasó?
La ventana, al intentar correr todo por medio del *thread* principal, no puede procesar eventos, como apretar un botón, mientras que está corriendo el *loop*.

#### Veamos como solucionarlo utilizando QThread

In [ ]:
class ThreadBacan(QThread):
    def __init__(self, actualizar_label_signal, *args, **kwargs):
        # Entregar *args y **kwargs a la super clase es importante por si queremos dar algun parametro
        # inicial de los que ya ofrece la clase QThread
        super().__init__(*args, **kwargs)
        # Le entregamos una senal que queremos que el Thread emita
        self.actualizar_label_signal = actualizar_label_signal

    def run(self):
        for _ in range(10):
            self.actualizar_label_signal.emit()
            sleep(0.5)

In [ ]:
class VentanaConThread(QWidget):
    actualizar_label_signal = pyqtSignal()

    def __init__(self):
        super().__init__()
        self.label_numero = QLabel("0", self)
        self.boton_numero = QPushButton("0", self)
        self.boton_loop = QPushButton("Iniciar Loop", self)

        self.layout_principal = QVBoxLayout(self)

        # Creamos nuestro thread y le entregamos la senal para actualizar el label
        self.thread_bacan = ThreadBacan(self.actualizar_label_signal)

        self.init_gui()

In [ ]:
    def init_gui(self):
        self.layout_principal.addWidget(self.label_numero)
        self.layout_principal.addStretch()
        self.layout_principal.addWidget(self.boton_numero)
        self.layout_principal.addWidget(self.boton_loop)

        self.boton_numero.clicked.connect(self.actualizar_boton)
        self.boton_loop.clicked.connect(self.iniciar_loop)
        self.actualizar_label_signal.connect(self.actualizar_label)

        self.show()

In [ ]:
    def actualizar_label(self):
        numero_actual = int(self.label_numero.text())
        self.label_numero.setText(str(numero_actual + 1))

    def actualizar_boton(self):
        numero_actual = int(self.boton_numero.text())
        self.boton_numero.setText(str(numero_actual + 1))

    def iniciar_loop(self):
        self.thread_bacan.start()

¡Threading puede ser muy útil!

Sin embargo, podría parecer tedioso tener que implementar un *thread* personalizado para todo lo que implique tiempo.
Es por esto que los ``QTimer`` pueden ser una herramienta muy poderosa, pues permite fácilmente enviar señales cada cierto tiempo (y nos ahorramos un poquito de código).

**Puedes encontrar la implementación con QTimer en los ejemplos de esta ayudantía**

## Main window 💻
### Qué es una MainWindow, ¿se come?
![imagen macewindu](imagenes/mace_windu_star_wars.jpg)

Cuando hablamos de una ``MainWindow``, piensa simplemente en una *widget* especial, la cual trae un orden pre-definido y funcionalidades especiales.
Esta ventana existe principalmente para facilitar la construcción de aplicaciones con un orden "estándar". Básicamente, te permite construir rápidamente aplicaciones que
ordenan sus ventanas como el *IDE* que utilizas para el ramo.

Una MainWindow se ordena de la siguiente manera:
![imagen mainwindow](imagenes/mainwindowlayout.png)

## La salvación de muchos: Qt Designer 🎨🙌
Qt Designer es una herramienta de diseño que permite crear Widgets visualmente 😎

### ¿Cómo lo encuentro? ¿Cómo lo uso en mi programa? ¡Ayuda!
Instalando designer:

``pip install PyQt5-tools``

``pip3 install PyQt5-tools``

Encontrando designer:

``C:\Users\[Tu usuario]\AppData\Local\Programs\Python\Python[version]\Lib\site-packages\pyqt5-tools\designer``

También puedes utilizar en consola el comando:

``pyqt5-tools designer``

![imagen mainwindow](imagenes/qtdesigner-anotado.png)

## Ejercicio propuesto: Preparandose para Halloween
El ejercicio consta utilizar tanto QtDesigner como Python para implementar una ventana. El proceso
que vamos a seguir es el siguiente:

 - Crearemos las ventanas utilizando Designer.
 - Conectaremos la señal de un botón utilizando Designer.
 - Importaremos el trabajo hasta este punto a Python.
 - Utilizaremos python para crear funcionalidades más complejas.